## Segmentation on lice-crop

Running segmentation on all the crops

In [ ]:
!nvidia-smi

First, let's list all the crops with lice

In [ ]:
import glob
import json
import os 

from tqdm import tqdm

In [ ]:
name2json = {}
all_crops = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/*/*/*.jpg")
for crop in all_crops:
    name2json[os.path.basename(crop)] = crop.replace("jpg", "json")

In [ ]:
# all_images = glob.glob("/root/data/lice-data/selected_crops/*.jpg")
# for img in all_images:
#     new_name = img.replace(" (copy)", "")
#     os.rename(img, new_name)

In [ ]:
all_images = glob.glob("/root/data/lice-data/selected_crops/*.jpg")
images_with_lice = []
for imgpath in tqdm(all_images):
    jsonpath = name2json[os.path.basename(imgpath)]
    detections = json.load(open(jsonpath))["detections"]
    if detections is None:
        continue
    if len(detections) > 0:
        images_with_lice.append(imgpath)
print("All crops: {}".format(len(all_images)))
print("Crops with lice {}".format(len(images_with_lice)))

In [ ]:
with open("./selected_crops_with_lice.json", "w") as f:
    json.dump(images_with_lice, f)

Predictions. 
gotta be smart about it. batch and shit

In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model

from unet import jaccard_coef_loss, jaccard_coef_int

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
new_shape = (768, 256)

In [ ]:
model_path = "/root/data/models/single_fish_unet_segmentation.h5"
model = load_model(model_path, 
                   custom_objects={'jaccard_coef_loss': jaccard_coef_loss,
                                   'jaccard_coef_int': jaccard_coef_int})

In [ ]:
batch = []
all_preds = []
for path in tqdm(images_with_lice):
    image = cv2.imread(path)
    image = cv2.resize(image, new_shape)
    batch.append(image)
    if len(batch) == 8:
        batch = np.array(batch)
        predictions = model.predict_on_batch(batch)
        all_preds.append(predictions)
        batch = []

In [ ]:
len(all_preds)

In [ ]:
c = 0
# good_mask = []
print(len(good_mask))
for i in range(10, 20, 1):
    predictions = all_preds[i]
    for j in range(8):
        plt.imshow(predictions[j, ...].squeeze(), alpha=0.5)
        plt.show()
        keep = int(input("Keep or not ") or 1)
        if keep:
            good_mask.append(images_with_lice[c])
        # print(images_with_lice[c])
        c += 1

In [ ]:
with open("./good_masks.json", "w") as f:
    json.dump(good_mask, f)